1. In Cardconvert, convert all binary files in Array type CSVs.

In [2]:
%matplotlib inline
import os
import sys
import platform
import matplotlib
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as tick
from matplotlib.backends.backend_pdf import PdfPages
import datetime
from pylab import rcParams

import seaborn as sns

import scipy
import pyproj
import folium

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot,iplot_mpl,plot_mpl
import plotly.graph_objs as go

init_notebook_mode(connected=True)

#import wellapplication as wa

#import geostatsmodels
import scipy.stats

import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import statsmodels.api as sm

from scipy.interpolate import griddata
#import cartopy
#import cartopy.crs as ccrs

import geopandas
from shapely.geometry import Point

from pyproj import CRS
from pyproj import Transformer



rcParams['figure.figsize'] = 15, 10

In [15]:
import pathlib

In [13]:
from ..MicroMet import micromet

mmpath = pathlib.Path("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/Data_Processing/Jupyter_Notebooks/MicroMet")
sys.path.append(llpath)
import loggerloader as ll

ImportError: attempted relative import with no known parent package

In [5]:
import importlib.util
spec = importlib.util.spec_from_file_location("micromet", "G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/Data_Processing/Jupyter_Notebooks/MicroMet/micromet/")
micromet = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dbconnect)



AttributeError: 'NoneType' object has no attribute 'loader'

In [3]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Pandas Version " + str(pd.__version__))
print("Numpy Version " + str(np.__version__))
print("Matplotlib Version " + str(matplotlib.__version__))
#print("Well Application Version " + str(wa.__version__))
print("Scipy Version " +str(scipy.__version__))
print (os.environ['CONDA_DEFAULT_ENV'])

Operating System Windows 10
Python Version 3.10.4 | packaged by conda-forge | (main, Mar 24 2022, 17:34:17) [MSC v.1929 64 bit (AMD64)]
Pandas Version 1.4.2
Numpy Version 1.21.6
Matplotlib Version 3.5.1
Scipy Version 1.8.0
etqaqc


# Useful Libraries (R and Python)

## Data Prep and Processing
* https://github.com/adamhsparks/EddyCleanR - Fills gaps and removes outliers in eddy covariance data.
* https://github.com/bgctw/REddyProc - Processing data from micrometeorological Eddy-Covariance systems 
* https://github.com/June-Spaceboots/EddyCovarianceProcessing - A collection of code used to processess Eddy Covaraince
* https://github.com/UofM-CEOS/flux_capacitor - Tools for processing flux data (eddy covariance).
* https://github.com/wsular/EasyFlux-DL-CR3000 - CR3000 datalogger program for Campbell open-path eddy-covariance systems
* https://github.com/Open-ET/flux-data-qaqc - Energy Balance Closure Analysis and Eddy Flux Data Post-Processing
* https://github.com/lsigut/openeddy - The R Package for Low Frequency Eddy Covariance Data Processing


## Partitioning Fluxes
* https://github.com/usda-ars-ussl/fluxpart - Python module for partitioning eddy covariance flux measurements. 
* https://github.com/jnelson18/ecosystem-transpiration - Code and examples of how to estimate transpiration from eddy covariance data. 

## Remote Sensing Estimates
* https://github.com/NASA-DEVELOP/METRIC - For estimating daily evapotranspiration from Landsat data 
* https://github.com/kratzert/pyTSEB - two Source Energy Balance model for estimation of evapotranspiration with remote sensing data 
* https://github.com/spizwhiz/openet-ssebop-beta - Earth Engine SSEBop ET Model 
* https://github.com/pblankenau2/openet-core-beta - OpenET Core Components 
* https://github.com/tomchor/pymicra - A Python tool for Micrometeorological Analysis

## MISC
* https://github.com/sunxm19/Planetary_boundary_height_for_FFP - extracting boundary height within North America from NOAA regional data

## Reference ET
* https://github.com/woodcrafty/PyETo - Python package for calculating reference/potential evapotranspiration (ETo).
* https://github.com/cgmorton/RefET-GEE - ASCE Standardized Reference Evapotranspiration Functions for Google Earth Engine (GEE) 
* https://github.com/usbr/et-demands - Dual crop coefficient crop water demand model 


# Relevant references

[Mauder, M., Cuntz, M., Drüe, C., Graf, A., Rebmann, C., Schmid, H.P., Schmidt, M., and Steinbrecher, R., 2013, A strategy for quality and uncertainty assessment of long-term eddy-covariance measurements: Agricultural and Forest Meteorology, v. 169, p. 122–135., doi: 10.1016/j.agrformet.2012.09.006.](https://drive.google.com/file/d/1u51mxImadycZJRp6fJwpG9j4utXVzUJm/view?usp=sharing)


[Foken, T., and Wichura, B., 1996, Tools for quality assessment of surface-based flux measurements: Agricultural and Forest Meteorology, v. 78, no. 1–2, p. 83–105., doi: 10.1016/0168-1923(95)02248-1.](https://drive.google.com/file/d/18oWqfbVnJ97jz7CFITh7Ul-GICV_xktA/view?usp=sharing)

# QA/QC Raw Data

* How do we decide what motions to count as eddy flux and what as
slow variations to be treated deterministically or as instrumental
drift to be discarded?
* What are the best tools to separate slowly changing (mean) and
rapidly varying ( turbulent r parts of our time series?
* Can we distinguish between instrument drift and low frequency
meteorological signal in the data?
* If we cannot distinguish, how large an error do we make if we
discard some low frequency meteorological signal along with instrument
drift?

the most important term was 'rotation and averaging', giving rise to 10-25% uncertainty in their estimates

* Averaging - best period to average the data over (default is 30 minutes)
* Detrending - removing low frequency instrument drift - typically linear, but can be nonlinear
* Filtering - removing spikes and noise; Low pass

The purpose of averaging, detrending and filtering is to separate the active turbulent transport that we treat as eddy flux from slower, deterministic atmospheric motions and instrument drift.

## Despiking

## Spectral Analysis

### Averaging Period

This can be selected using one of two ways:
* Cospectral forms (FT of data)
* Ogive plots that integrate under the cospectral curve ([Pymicra](https://pymicra.readthedocs.io/en/latest/tutorial.html?highlight=ogive#obtaining-the-spectra) does this)

https://github.com/BSchilperoort/hydrproc/blob/master/exercise.1.eddy_covariance.ipynb 

https://github.com/cokelaer/spectrum

https://nbviewer.jupyter.org/github/mattijn/pynotebook/blob/master/2016/2016-05-25%20cross-spectral%20analysis.ipynb

## Time Lag Determination

Spectral attenuation

## Range Checks

## Filtering Tools

* Finite Impulse Response FIR - [Scipy Signal Tool](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html)
* Recursive Filter - [Statsmodel Tool](https://www.statsmodels.org/dev/generated/statsmodels.tsa.filters.filtertools.recursive_filter.html)
* RC Filter - [Notebook Example](https://notebook.community/mholtrop/Phys605/Python/Signal/RC_Filters_in_Python)

https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units

## Coordinate Rotation

The [Fluxer](https://github.com/UofM-CEOS/flux_capacitor/blob/master/docs/source/index.rst) package does coordinate rotations

### Traditional

In [4]:
def coord_rotation(df, Ux='Ux', Uy='Uy', Uz='Uz'):
    """Traditional Coordinate Rotation
    """
    xmean = df[Ux].mean()
    ymean = df[Uy].mean()
    zmean = df[Uz].mean()
    Uxy = np.sqrt(xmean ** 2 + ymean ** 2)
    Uxyz = np.sqrt(xmean ** 2 + ymean ** 2 + zmean ** 2)
    cosν = xmean / Uxy
    sinν = ymean / Uxy
    sinTheta = zmean / Uxyz
    cosTheta = Uxy / Uxyz
    return cosν, sinν, sinTheta, cosTheta, Uxy, Uxyz

In [ ]:

def rotation12_momentum(alph, gamm, U_mean, V_mean, W_mean, UU_cov, VV_cov, WW_cov, UVcov, UWcov, VWcov):
    """Coordinate rotation corrections.  This function applies the first and second rocations in Tanner and Thurtell (1969).
    a. Counterclockwise about the instrument z-axis for Gamma degrees
    b. Counterclockwise about the intermediate y-axis for Alpha degrees (Originally clockwise, but this function follows Wilczak and 
    others (2001) convention of counterclockwise rotation)
    
    This function transforms the expression of momentum variables in the instrument coordinate system to the natural wind coordinate 
    system.
    
    alph = alpha; output of CSAT unit; Angle of counterclockwise rotation about y-axis (pitch); decimal degrees; Wilczak and others (2001)
    gamm = gamma; output of CSAT unit; Angle of counterclockwise rotation about z-axis (yaw); decimal degrees; Wilczak and others (2001)
    U_mean = Ux_Avg; Average Ux wind vector from CSAT unit; cov_out_sonic(1); (meters per second)
    V_mean = Uy_Avg; Average Uy wind vector from CSAT unit; cov_out_sonic(5); (meters per second)
    W_mean = Uz_Avg; Average Uz wind vector from CSAT unit; cov_out_sonic(8); (meters per second)
    UU_cov = Ux_SIGMA; standard deviation of Ux wind vector from CSAT unit; cov_out_sonic(2); (meters per second)
    VV_cov = Uy_SIGMA; standard deviation of Uy wind vector from CSAT unit; cov_out_sonic(6); (meters per second)
    WW_cov = Uz_SIGMA; standard deviation of Uz wind vector from CSAT unit; cov_out_sonic(9); (meters per second)
    UVcov = UxUy_cov; covariance of Ux and Uy; cov_out_sonic(3); (m2 per s2)
    UWcov = UxUz_cov; covariance of Ux and Uz; cov_out_sonic(4); (m2 per s2)
    VWcov = UyUz_cov; covariance of Uy and Uz; cov_out_sonic(7); (m2 per s2)

    """
 
    # UWcov_R and VWcov_R are used first as intermediate variables before both variables are needed for their own roles.
    UWcov_R =  UU_cov*np.cos(gamm)*np.cos(gamm) + VV_cov*np.sin(gamm)*np.sin(gamm)
    VWcov_R =  UWcov*np.cos(gamm) + VWcov*np.sin(gamm)
    
    # 3a. ROTATIONS FOR MEAN TERMS
    Umean_R = np.cos(alph)*(U_mean*np.cos(gamm) + V_mean*np.sin(gamm)) - W_mean*np.sin(alph)
    Vmean_R = 0
    Wmean_R = np.sin(alph)*(U_mean*np.sin(gamm) + V_mean*np.sin(gamm)) + W_mean*np.cos(alph)

    # b. ROTATIONS FOR VARIANCE TERMS
    UUcov_R = np.cos(alph)*np.cosCOS(alph)*UWcov_R + WW_cov*np.sin(alph)*np.sin(alph) + UVcov*np.cos(alph)*np.cos(alph)*np.sin(2*gamm) - np.sin(2*alph)*VWcov_R

    VVcov_R = UU_cov*np.sin(gamm)*np.sin(gamm) + VV_cov*np.cos(gamm)*np.cos(gamm) - UVcov*np.sin(2*gamm)

    WWcov_R = np.sin(alph)*np.sin(alph)*UWcov_R + WW_cov*np.cos(alph)*np.cos(alph) + UVcov*np.sin(alph)*np.sin(alph)*np.sin(2*gamm) +   SIN(2*alph)*VWcov_R

    # c. ROTATIONS FOR COVARIANCE TERMS
    UVcov_R = -0.5*(UU_cov - VV_cov)*np.cos(alph)*np.sin(2*gamm) + UVcov*np.cos(alph)*np.cos(2*gamm) + np.sin(alph)*(UWcov*np.sin(gamm) - VWcov*np.cos(gamm))

    UWcov_R =  0.5*np.sin(2*alph)*(UWcov_R - WW_cov + UVcov*np.sin(2*gamm)) + np.cos(2*alph)*VWcov_R

    VWcov_R = -np.sin(alph)*(0.5*(UU_cov - VV_cov)*np.sin(2*gamm)-UVcov*np.cos(2*gamm)) - np.cos(alph)*(UWcov*np.sin(gamm) - VWcov*np.cos(gamm))

    return UVcov_R, UWcov_R, VWcov_R, UUcov_R, VVcov_R, WWcov_R


def rotation12_scalar_covariance(alph, gamm, SU_cov, SV_cov, SW_cov):
    """The 1 and 2 in the function indicates the 1st and 2nd rotations in Tanner and Thurtell (1969). 
    This function transform the expression of variance of scalar with momentum variables in the instrument 
    coordinate system to the natural wind coordinate system in x and y.
    S is a scalar variable (e.g. CO2, H2O, or T_SONIC)
    U_cov, V_cov, W_cov 
    
    alph = alpha; output of CSAT unit; Angle of counterclockwise rotation about y-axis (pitch); decimal degrees; Wilczak and others (2001)
    gamm = gamma; output of CSAT unit; Angle of counterclockwise rotation about z-axis (yaw); decimal degrees; Wilczak and others (2001)

    
    """

    SUcov_R =  np.cos(alph)*(SU_cov*np.cos(gamm) + SV_cov*np.sin(gamm)) - SW_cov*np.sin(alph)
    SVcov_R = -SU_cov*np.sin(gamm) + SV_cov*np.cos(gamm)
    SWcov_R =  np.sin(alph)*(SU_cov*np.cos(gamm) + SV_cov*np.sin(gamm)) + SW_cov*np.cos(alph)

    return SUcov_R, SVcov_R, SWcov_R



### Planar Fit

* Determine a period (weeks or longer) during which there was no change in the anemometer's position relative to the surface.
* Perform linear regression, using data from the chosen period to define a "tilted plane" , or the mean streamline plane (Figure 3.5), where bo, b1 and b2 are regression coefficients, and {u1, 'iJ1, w1} are components of the (30-min) mean velocity in the instrument coordinate system.
* Use the regression coefficient b1 and b2 to determine the pitch, roll and yaw angles for rotation as in Wilczak et al. (2001) or alternatively, the base vector set that defines the three coordinate axes (Appendix B).
* Project the velocity and flux cross products into the new coordinate system.

# Variance and Covariance Computation